In [1]:
import os
from os.path import join
from IPython.display import Image, display
import numpy as np
import tempfile
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation,MaxPooling2D,Dropout,BatchNormalization,AveragePooling2D,Input,ZeroPadding2D,Add


In [2]:
#data prep
train_path = 'C:\\Users\\Danial\\Documents\\chest_xray\\train'
val_path = 'C:\\Users\\Danial\\Documents\\chest_xray\\val'
test_path = 'C:\\Users\\Danial\\Documents\\chest_xray\\test'
train_batch = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        validation_split=0.2,
        ).flow_from_directory(train_path,target_size = (100,100),classes =['NORMAL','PNEUMONIA'] ,batch_size=34,color_mode="rgb",subset="training")
test_batch = ImageDataGenerator(rescale=1/255).flow_from_directory(test_path,target_size = (100,100),classes =['NORMAL','PNEUMONIA'] ,color_mode="rgb")
val_batch = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
        ,validation_split=0.2).flow_from_directory(train_path,target_size = (100,100),classes =['NORMAL','PNEUMONIA'],batch_size=4,color_mode="rgb",subset="validation")

Found 4199 images belonging to 2 classes.
Found 644 images belonging to 2 classes.
Found 1049 images belonging to 2 classes.


In [3]:
#Model structure Definitions
def convblock(X, f, filters):
    #Filter
    F1,F2,F3 = filters
    # Save the input value
    X_shortcut = X


    #MAIN PATH
    # First 
    X = Conv2D(F1, (1, 1), strides = (2,2))(X)
    X = Dropout(0.2)(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second 
    X = X = Conv2D(F2, (f, f), strides = (1,1),padding = 'same',kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dropout(0.2)(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third 
    X = X = Conv2D(F3, (1, 1), strides = (1,1),padding='valid',kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dropout(0.2)(X)
    X = BatchNormalization(axis=3)(X)

    
    #SHORTCUT PATH
    X_shortcut = Conv2D(F3, (1, 1), strides = (2,2),kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X = Dropout(0.2)(X)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)

    # add shortcut
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [4]:
def idblock(X, f, filters):
    #filters
    F1,F2,F3=filters
    X_shortcut = X

    # First part
    X = Conv2D(F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second part
    X = Conv2D(F2, kernel_size = (f, f), strides = (1,1), padding = 'same',kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dropout(0.2)(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third part
    X = Conv2D(F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid',kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dropout(0.2)(X)
    X = BatchNormalization(axis = 3)(X)

    # Shortcut
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [5]:
def myres(shape,classes):
    #Inputs
    X_input = Input(shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convblock(X, f = 3, filters = [16, 16, 64])
    X = idblock(X, 3, [16, 16, 64])
    X = idblock(X, 3, [16, 16, 64])

    # Stage 3
    X = convblock(X, f = 3, filters = [32, 32, 128])
    X = idblock(X, 3, [32, 32, 128])
    X = idblock(X, 3, [32, 32, 128])
    X = idblock(X, 3, [32, 32, 128])

    # Stage 4
    X = convblock(X, f = 3, filters = [64, 64, 256])
    X = idblock(X, 3, [64, 64, 256])
    X = idblock(X, 3, [64, 64, 256])
    X = idblock(X, 3, [64, 64, 256])
    X = idblock(X, 3, [64, 64, 256])
    X = idblock(X, 3, [64, 64, 256])

    # Stage 5
    X = convblock(X, f = 3, filters = [128, 128, 512])
    X = idblock(X, 3, [128, 128, 512])
    X = idblock(X, 3, [128, 128, 512])

    # AVGPOOL.
    X = AveragePooling2D((2, 2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(512,activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax')(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='myres')

    return model

In [6]:
my_net = myres(train_batch.image_shape,2)
my_net.summary()
my_net.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=['accuracy'])

Model: "myres"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 106, 106, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 50, 50, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 50, 50, 64)   256         conv2d[0][0]                     
______________________________________________________________________________________________

In [7]:
checkpoint = ModelCheckpoint("./val", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
my_net.fit(train_batch,epochs = 50,callbacks=callbacks_list,validation_data = val_batch)

Epoch 1/50
124/124 [==============================] - ETA: 0s - loss: 0.7107 - accuracy: 0.7097
Epoch 00001: val_accuracy improved from -inf to 0.74166, saving model to ./val
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./val\assets
124/124 [==============================] - 152s 1s/step - loss: 0.7107 - accuracy: 0.7097 - val_loss: 0.7731 - val_accuracy: 0.7417
Epoch 2/50
124/124 [==============================] - ETA: 0s - loss: 0.4305 - accuracy: 0.7833
Epoch 00002: val_accuracy did not improve from 0.74166
124/124 [==============================] - 98s 794ms/step - loss: 0.4305 - accuracy: 0.7833 - val_loss: 1.5909 - val_accuracy: 0.7417
Epoch 3/50
124/124 [==============================] - ETA: 0s - loss: 0.3294 - accuracy: 0.8557
Epoch 00003: val_accuracy did not improve from 0.74166
124/124 [==============================] - 99s 801ms/step - loss: 0.3294 - accuracy: 0.8557 - val_loss: 1.7608 - val_accuracy: 0

In [10]:
y_img_batch, y_class_batch = test_batch[0] 
y_pred = np.argmax(my_net.predict(y_img_batch),-1)
y_true = np.argmax(y_class_batch,-1)

print(y_true.shape)
print(classification_report(y_true, y_pred))

(32,)
              precision    recall  f1-score   support

           0       1.00      0.55      0.71        11
           1       0.81      1.00      0.89        21

    accuracy                           0.84        32
   macro avg       0.90      0.77      0.80        32
weighted avg       0.87      0.84      0.83        32



In [11]:
evalu = my_net.evaluate(test_batch)
print (evalu)

21/21 [==============================] - 8s 364ms/step - loss: 0.7780 - accuracy: 0.8152
[0.7780068516731262, 0.8152173757553101]
